In [1]:
import pandas as pd
import math
import os
from datetime import datetime
import xarray as xr

# Listado de playas con avistamientos y fechas

In [10]:
avistamientos_df = pd.read_excel("./Physalia_Ambiental_R.xlsx")
# avistamientos_df = pd.read_excel("../Physala_Data/Datos_Physalia_20171010.xls")

columnas = avistamientos_df.iloc[0]

#Quito las 3 primeras filas, debido al formato de la excell
avistamientos_df = avistamientos_df.iloc[3:] 
avistamientos_df.columns = columnas

# Me quedo solo con los datos de avistamientos
avistamientos_df = avistamientos_df[["Latitud","Longitud","Año","Mes","Día","Avistamientos"]]

#Transdormaciones para sacar con fecha (datetime)
avistamientos_fecha_df=avistamientos_df[["Año","Mes","Día"]]
avistamientos_fecha_df.columns = ["year","month","day"]

fecha = pd.to_datetime(avistamientos_fecha_df)

avistamientos_df["Fecha"]=fecha
avistamientos_df=avistamientos_df[["Latitud","Longitud","Fecha","Avistamientos"]]
avistamientos_df.head()

,Latitud,Longitud,Fecha,Avistamientos
3,-25.4,-70.4833,2014-05-01,1
4,-18.4631,-70.3053,2014-05-01,1
5,-27.0658,-70.8259,2014-05-09,13
6,-18.4933,-70.3261,2014-05-10,15
7,-27.122,-70.8576,2014-05-10,23


# Exporto listado de playas para web

In [11]:
# Genera excel con las coordenadas de las playas
playas_df = avistamientos_df.reset_index()
playas_df = playas_df[['Latitud','Longitud']]
playas_df.to_excel('./playas.xlsx')

# Redondeo
Redondeo de latitud y longitud para juntar lecturas de una misma playa con coordenadas muy proximas

Se exporta el dataframe generado a un excel

In [12]:
def floorQuarter(x):
    return (floor(x * 4) / 4.0) 

#Se añaden atributos con la longitud y latitud redondeadas a cada cuarto de grado
avistamientos_df["Lat_floor"] = avistamientos_df.Latitud.map(floorQuarter)
avistamientos_df["Long_floor"] = avistamientos_df.Longitud.map(floorQuarter)

df = avistamientos_df[['Lat_floor', 'Long_floor',"Fecha","Avistamientos"]]
df = df.set_index(['Lat_floor', 'Long_floor',"Fecha"])

#DataFrame total avistamientos de playas en esa cuadricula
df_sum = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).sum()

#DataFrame número de playas en esa cuadricula
df_count = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).count()

#Dataframe con el total de avistamientos y el número de playas
df_join = df_sum.join(df_count,lsuffix="I",rsuffix="R")
df_join.columns=['Suma', 'N_Playas']
df_join['Media_Playas'] = df_join.Suma /df_join.N_Playas

# DataFrame de avistamientos procesado
df_join.to_excel("avistamientos.xlsx")

df_join.head()

Suma  N_Playas  Media_Playas
Lat_floor Long_floor Fecha                                   
-42.75    -74.25     2015-05-27     1         1           1.0
-42.00    -74.00     2014-06-04     1         1           1.0
          -73.75     2015-05-23     1         1           1.0
-41.75    -73.75     2015-06-10    20         1          20.0
                     2015-06-11   151         1         151.0

# Estructura Buena

In [17]:
# DF con playas
df_playas = pd.read_excel('./avistamientos.xlsx').reset_index()
'''
Las celdas que estaban unidas por tener el mismo valor, tienen valores nulos
por lo que las relleno con us valor correspodiente
'''
df_playas = df_playas.fillna(method= 'ffill',axis=0)
# Cojo las columnas que me interesan
df_playas = df_playas[["Suma","Lat_floor","Long_floor","Fecha"]]
# Renombro las columnas 
df_playas = df_playas.rename(columns={"Suma": "Avistamientos", "Lat_floor": "Latitud", "Long_floor": "Longitud"})
# Ordeno por fecha
df_playas = df_playas.sort_values(by=["Fecha"])

In [18]:
df_playas # DataFrame con avistamientos en cada playa
cuadrantes = { # Numero de cuadrantes que se concatenarán con las playas
    'izquierda': 2 ,
    'arriba': 1,
    'abajo': 1
}
salto = 1/12 # Salto de coordenadas entre los cuadrantes
listado_archivos = os.listdir('../descargas') # Listo todos los archivos de Copernicus
variables = ['Coord','Profundidad','mlotst','zos','bottomT','thetao','so','uo','vo']

In [19]:
def busca_archivo(fecha):
    texto ='_{}_'.format(str(fecha).split()[0].replace('-',''))
    archivo = [x for x in listado_archivos if str(texto) in x]
    data = xr.open_dataset('../descargas/{}'.format(archivo[0])) # cargo el archivo
    return data # devuelvo dataset

def dame_datos(latitud,longitud,ds):
    return ds.sel({'latitude':latitud,'longitude': longitud})

def comprueba_datos(latitud,longitud,ds):
    valor = dame_datos(latitud,longitud,ds)
    while math.isnan(valor.mlotst[0]):
        longitud = longitud - salto
        valor = dame_datos(latitud,longitud,ds)
    return latitud,longitud # devuelvo las coordenadas con datos

def crea_DF():
    df_resultados = None
    df_resultados = pd.DataFrame(columns=["Avistamientos","Latitud","Longitud","Fecha",'Coord',"Profundidad","mlotst",'zos','bottomT','thetao','so','uo','vo']) # columnas básicas
    for izquierda in range(cuadrantes['izquierda']): # por cada cuadrante a la izquierda meto un grupo de columnas
        var_aux = [str(x+'_izq_'+str(izquierda+1)) for x in variables]
        df_aux = pd.DataFrame(columns=var_aux)
        df_resultados = df_resultados.join(df_aux)
        for superior in range (cuadrantes['arriba']): # por cada cuadrante a la superior meto un grupo de columnas
            var_aux = [x+'_sup_'+str(izquierda+1)+'.'+str(superior+1) for x in variables]
            df_aux = pd.DataFrame(columns=var_aux)
            df_resultados = df_resultados.join(df_aux)
        for inferior in range (cuadrantes['abajo']): # por cada cuadrante a la inferior meto un grupo de columnas    
            var_aux = [x+'_inf_'+str(izquierda+1)+'.'+str(inferior+1) for x in variables]
            df_aux = pd.DataFrame(columns=var_aux)
            df_resultados = df_resultados.join(df_aux)
    return df_resultados

def meto_en_df(df_resultados,row,coordenadas,df,indice,variables_actuales):
    for x in range(3):
        df_resultados.loc[indice + x,["Avistamientos","Latitud","Longitud","Fecha"]] = row.values.tolist() # meto en df_resulados los datos de ese cuadrante
        vars = df.loc[x,['depth', 'mlotst','zos','bottomT','thetao','so','uo','vo']].values.tolist()
        vars.insert(0,coordenadas)
        df_resultados.loc[indice + x,variables_actuales] = vars # meto coordenadas reales utilizadas y datos 
    return df_resultados

def dameCoordIzq(lat,long,n_saltos):
    return [lat,long-(salto*n_saltos)]

def dameCoordSuperior(lat,long,n_saltos):
    coordenadas = dameCoordIzq(lat,long,n_saltos)
    return [lat+(salto*n_saltos),long]

def dameCoordInferior(lat,long,n_saltos):
    return [lat-(salto*n_saltos),long]

In [20]:
def crea_estr():
    df_resultados = crea_DF() # creo df con columnas necesarias
    #print(df_resultados)
    for contador,[index, row] in enumerate(df_playas.iterrows()): # por cada playa
        print(contador,end='\r')
        ds = busca_archivo(row['Fecha']) # busco el archivo de la fecha 
        coordenadas = comprueba_datos(row['Latitud'],row['Longitud'],ds) # miro a ver si el cuadrante tiene datos, sino cojo el siguiente por la izquierda

        df = dame_datos(coordenadas[0],coordenadas[1],ds).to_dataframe().reset_index() # dataframe con los datso oceanicos de esas coordenadas y en esa fecha
        df_resultados = meto_en_df(df_resultados,row,coordenadas,df,contador * 3,variables)

        # datos de las celdas extras
        for izquierda in range(cuadrantes['izquierda']): # por cada cuadrante a la izquierda, añado datos
            coordenadas = dameCoordIzq(coordenadas[0],coordenadas[1],izquierda+1)
            df = dame_datos(coordenadas[0],coordenadas[1],ds).to_dataframe().reset_index() 
            var_aux = [x+'_izq_'+str(izquierda+1) for x in variables]
            df_resultados = meto_en_df(df_resultados,row,coordenadas,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for superior in range (cuadrantes['arriba']): # por cada cuadrante a la superior, añado datos
                coord_aux = dameCoordSuperior(coord_aux[0],coord_aux[1],superior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_sup_'+str(izquierda+1)+'.'+str(superior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for inferior in range (cuadrantes['abajo']): # por cada cuadrante a la inferior, añado datos  
                coord_aux = dameCoordInferior(coord_aux[0],coord_aux[1],inferior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_inf_'+str(izquierda+1)+'.'+str(inferior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)

#         if contador == 5:
#             break
    return df_resultados

In [21]:
df_resultados = crea_estr()
df_resultados

,Avistamientos,Latitud,Longitud,Fecha,Coord,Profundidad,mlotst,zos,bottomT,thetao,...,vo_sup_2.1,Coord_inf_2.1,Profundidad_inf_2.1,mlotst_inf_2.1,zos_inf_2.1,bottomT_inf_2.1,thetao_inf_2.1,so_inf_2.1,uo_inf_2.1,vo_inf_2.1
0,1,-18.5,-70.5,2014-05-01 00:00:00,"(-18.5, -70.5)",0.494025,10.5289,0.222785,13.3423,18.6518,...,-0.0567644,"[-18.583333333333332, -70.75]",0.494025,13.2756,0.209967,4.45994,21.1773,35.1726,0.0897244,-0.162358
1,1,-18.5,-70.5,2014-05-01 00:00:00,"(-18.5, -70.5)",5.07822,10.5289,0.222785,13.3423,18.528,...,-0.0842311,"[-18.583333333333332, -70.75]",5.07822,13.2756,0.209967,4.45994,21.156,35.1711,0.093997,-0.189215
2,1,-18.5,-70.5,2014-05-01 00:00:00,"(-18.5, -70.5)",9.573,10.5289,0.222785,13.3423,18.3727,...,-0.102542,"[-18.583333333333332, -70.75]",9.573,13.2756,0.209967,4.45994,21.1289,35.1695,0.0958281,-0.202643
3,1,-25.5,-70.5,2014-05-01 00:00:00,"(-25.5, -70.58333333333333)",0.494025,10.6815,0.198675,13.1834,15.2078,...,-0.151372,"[-25.583333333333332, -70.83333333333333]",0.494025,13.4281,0.162664,4.91992,15.4415,34.7301,-0.16419,-0.122684
4,1,-25.5,-70.5,2014-05-01 00:00:00,"(-25.5, -70.58333333333333)",5.07822,10.6815,0.198675,13.1834,15.1155,...,-0.178228,"[-25.583333333333332, -70.83333333333333]",5.07822,13.4281,0.162664,4.91992,15.399,34.7301,-0.169683,-0.150761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,5,-27,-71,2016-03-13 00:00:00,"(-27.0, -71.0)",5.07822,10.5289,0.0964385,7.34724,20.8433,...,0.0836207,"[-27.083333333333332, -71.25]",5.07822,10.5289,0.0921659,1.3954,20.0515,34.6583,0.122684,0.0677511
1310,5,-27,-71,2016-03-13 00:00:00,"(-27.0, -71.0)",9.573,10.5289,0.0964385,7.34724,20.313,...,0.0695822,"[-27.083333333333332, -71.25]",9.573,10.5289,0.0921659,1.3954,19.7717,34.6477,0.130619,0.044557
1311,2,-26.5,-70.75,2016-03-13 00:00:00,"(-26.5, -70.75)",0.494025,10.5289,0.0680563,13.0881,19.9145,...,0.269173,"[-26.583333333333332, -71.0]",0.494025,11.597,0.128483,4.43138,22.3587,34.8476,0.0268563,0.26429
1312,2,-26.5,-70.75,2016-03-13 00:00:00,"(-26.5, -70.75)",5.07822,10.5289,0.0680563,13.0881,19.3454,...,0.242317,"[-26.583333333333332, -71.0]",5.07822,11.597,0.128483,4.43138,22.2869,34.8445,0.0201422,0.238655


In [22]:
df_resultados.to_excel('pruebasMasCuadrantes.xlsx')
